# Finetuning GPT2 on a Quotes Dataset
#### Dataset: https://github.com/ShivaliGoel/Quotes-500K



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import numpy as np

In [ ]:
#Load Dataset
df = pd.read_csv('drive/MyDrive/gpt2-quoter/quotes_dataset.csv')

df = df.iloc[:, 0:3]
df.columns = ['Quote', 'Author', 'Categories']
#
df.describe()

searchfor = ['inspiration', 'motivation', 'life-lesson', 'love', 'hope', 'friendship', 'life', 'faith', 'universe', 'nature']
df = df[df['Categories'].str.contains('|'.join(searchfor),na=False)]
df['Length'] = df['Quote'].str.split().apply(len)
df.describe()


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,26,29,31,36,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Length
count,103292.000000
mean,33.456115
std,41.138829
min,1.000000
25%,12.000000
50%,20.000000
75%,38.000000
max,730.000000


In [ ]:
train_test_ratio = 0.85
train_valid_ratio = 70/85
df_full_train, df_test = train_test_split(df, train_size = train_test_ratio, random_state = 1)
# df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

In [ ]:
!pip install transformers
!pip install contractions

from transformers import AutoTokenizer,AutoModelWithLMHead
import contractions



tokenizer = AutoTokenizer.from_pretrained("gpt2")

def prepare_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    quotes = df['Quote'].tolist()
    for quote in quotes:
        #Making all words lower case
        quote = str(quote).strip().lower()

        #Removing extra whitespaces
        quote = re.sub(r"\s", " ", quote)

        #Expand contractions
        quote = contractions.fix(quote)

        #Filtering out quotes longer than 35 words and less than 10
        if(len(quote.split())<10 or len(quote.split())>100):
          continue

        #Remove non-ASCII characters
        encoded_string = quote.encode("ascii", "ignore")
        quote = encoded_string.decode()

        #Remove string with these characters
        searchfor = ['~', '-', '--']
        match = False
        for s in searchfor:
          if(s in quote):
            match = True
            break
        if(match):
          continue

        #Insert whitespace between word and punctuation
        # print(quote)
        quote = re.sub(r"([\w/'+$\s-]+|[^\w/'+$\s-]+)\s*", r"\1 ", quote)
        quote = re.sub(r'<[^>]+>', r"",quote)
        # print(quote)
        data += tokenizer.special_tokens_map['bos_token']+quote+tokenizer.special_tokens_map['eos_token']+'\n'
        
    f.write(data)

     |████████████████████████████████| 1.5MB 10.7MB/s 
     |████████████████████████████████| 2.9MB 41.8MB/s 
     |████████████████████████████████| 890kB 36.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ad28ffd856384c10e321d56b3f96c928c9e876f01b610c52b0535e2a00242726
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 317kB 8.4MB/s 
     |████████████████████████████████| 245kB 17.0MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81701 sha256=7beef2307c0d29bff314185d408857a583ea9461edd521c26a320ae18e2a5363
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
prepare_dataset(df_full_train, 'train.txt')
prepare_dataset(df_test, 'test.txt')
# prepare_dataset(df_valid, 'valid.txt')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilgpt2') 
model = AutoModelWithLMHead.from_pretrained('./drive/MyDrive/gpt2-quoter/modelv1/checkpoint-30000')

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:890: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset('train.txt','test.txt',tokenizer)


/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (2389931 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./drive/MyDrive/gpt2-quoter/modelv1", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=15, # number of training epochs
    per_gpu_train_batch_size=32, # batch size for training
    per_gpu_eval_batch_size=64,  # batch size for evaluation
    logging_steps = 500, # Number of update steps between two evaluations.
    save_steps=5000, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    # prediction_loss_only=True,
    # compute_metrics=True
)

In [ ]:
!pip install hyperdash
!hyperdash login --email

from hyperdash import Experiment

exp = Experiment("Distil GPT2 Quotes Motivational 10-100, 50 epochs v1")
epochs = exp.param("Epochs", 50)
trainer.train()
trainer.save_model("./drive/MyDrive/gpt2-quoter/modelv1")
tokenizer.save_pretrained("./drive/MyDrive/gpt2-quoter/modelv1")
exp.end()

from google.colab import files
files.download('./drive/MyDrive/gpt2-quoter/modelv1/pytorch_model.bin') 



  Created wheel for hyperdash: filename=hyperdash-0.15.3-cp36-none-any.whl size=28553 sha256=b0ac87aa30dc25f992d682f646e8a5b158ef6da3cc695867c740f28ac15767f2
  Stored in directory: /root/.cache/pip/wheels/62/5f/af/bbcaeb6570e4904c14fb4c1b70fee559a3788182ce4d104ce7
Successfully built hyperdash
Email address: noelmathewisaac@gmail.com
Password: 
Successfully logged in! We also installed: zR87CyPghk5v0/4Ud7uTYQoEY4G9u4/Oakd3mao5aP0= as your default API key
{ Epochs: 50 }


Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


Step,Training Loss
500,2.992918
1000,2.974431
1500,2.946668
2000,2.911925
2500,2.888464
3000,2.865781
3500,2.842747
4000,2.816496
4500,2.799606
5000,2.781114


This run of Distil GPT2 Quotes Motivational 10-100, 50 epochs v1 ran for 3:04:47 and logs are available locally at: /root/.hyperdash/logs/distil-gpt2-quotes-motivational-10-100-50-epochs-v1/distil-gpt2-quotes-motivational-10-100-50-epochs-v1_2021-01-08t11-55-57-375210.log


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install transformers
from transformers import *

tokenizer = AutoTokenizer.from_pretrained("./drive/MyDrive/gpt2-quoter/modelv1")
model = AutoModelWithLMHead.from_pretrained("./drive/MyDrive/gpt2-quoter/modelv1")

gpt2_finetune = pipeline('text-generation', model=model, tokenizer=tokenizer)


/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:890: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
def generate_quote(starting_text, min_length, max_length ):
  gen_text = gpt2_finetune (starting_text, min_length = min_length, max_length= max_length, top_k=50, top_p=0.95, temperature=0.5)
  return gen_text[0]['generated_text']

# %rm -rf ./drive/MyDrive/gpt2-quoter/modelv1




In [ ]:
prompts = ['why am i here',
'i am cool and good ',
'how are thou',
'Life is a journey',
'In the end ',
'Happiness is '
]

all_quotes = []

for prompt in prompts:
  for i in range (10):
   all_quotes.append(generate_quote(prompt, 10, 50))


In [ ]:
for q in all_quotes:
  print(q)